# Multidimensional data in pandas

Files needed = ('CPS_March_2016.csv')

We have covered some pandas basics and learned how to plot. Now let's sort out how to deal with more complex data. We will often find ourselves with data in which the unit of observation is complex. Pandas helps us deal with this by allowing for many index variables. So far, we have only used single indexing, but that is about to change. 

Some examples that could use a multiIndex
1. State and country
2. Team and player
3. Industry and firm
4. Country (or person, firm,...) and time

That last one is important, and one that shows up a lot in economics. We call this *panel data*. Panel data is sometimes called longitudinal data. It follows the same firm/person/country over time. 

In [1]:
import pandas as pd                 # load pandas and shorten it to pd
import datetime as dt               # load datetime and shorten it to dt
import matplotlib.pyplot as plt     # for making figures

In [2]:
soccer = {'team' : ['Man City', 'Man City', 'Man City', 'Man City', 'Chelsea', 'Chelsea'], 
          'player' : ['Walker', 'Stones', 'Foden', 'Jesus', 'Cahill', 'Pedro'],
          'pos' : ['D', 'D', 'M', 'F', 'D', 'F'],
          'goals' : [1, 0, 0, 1, 0, 3],
          'assists': [0,0,0,0,0,0]
         }

prem = pd.DataFrame(soccer)
prem

,team,player,pos,goals,assists
0,Man City,Walker,D,1,0
1,Man City,Stones,D,0,0
2,Man City,Foden,M,0,0
3,Man City,Jesus,F,1,0
4,Chelsea,Cahill,D,0,0
5,Chelsea,Pedro,F,3,0


### Multiple indexing
The key to working with more complex datasets is getting the index right. So far, we have considered a single index, but pandas allows for multiple indexes that nest each other. 

**Key concept:** Hierarchical indexing takes multiple *levels* of indexes. 

Let's set up the DataFrame to take team and position as the indexes. 

In [3]:
prem.set_index(['team', 'pos'], inplace=True)
prem

player  goals  assists
team     pos                        
Man City D    Walker      1        0
         D    Stones      0        0
         M     Foden      0        0
         F     Jesus      1        0
Chelsea  D    Cahill      0        0
         F     Pedro      3        0

Wow. 

Notice that the `set_index()` method is the same one we used earlier with single indexes. In this case, we passed it a list of variables to make the indexes
```python
prem.set_index(['team', 'pos'], inplace=True)
```

In the output, the highest level of the index is team (we passed it 'team' first in the list) and the second level is position. The output does not repeat the team name for each observation. The 'missing' team name just means that the team is the same as above. \[A very Tufte-esque removal of unnecessary ink.\] 

Let's take a look under the hood. What's our index? A new kind of object: the MultiIndex

In [4]:
print(prem.index)

MultiIndex([('Man City', 'D'),
            ('Man City', 'D'),
            ('Man City', 'M'),
            ('Man City', 'F'),
            ( 'Chelsea', 'D'),
            ( 'Chelsea', 'F')],
           names=['team', 'pos'])


### Subsetting with multiple indexes
With a multi index, we need two arguments to reference observations. Notice that I am using a **tuple** to pass the two values of the multiIndex.

In [5]:
# All the defenders on Man City
prem.loc[('Man City', 'D'),:] 

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


player  goals  assists
team     pos                        
Man City D    Walker      1        0
         D    Stones      0        0

It's always a good idea to pay attention to warnings, particularly 'PerformanceWarning'. Pandas is telling us that we are asking for something in the second index, but the second index is not ordered. If the index was big, this could slow down our program. Let's fix that with `sort_index()`.

**Important** Sort your mulitIndex. 

In [6]:
# Tell pandas which axis to sort. We could sort the columns, too...
# This returns a new DataFrame unless we use inplace=True.
prem = prem.sort_index(axis=0)   
                                 
prem

player  goals  assists
team     pos                        
Chelsea  D    Cahill      0        0
         F     Pedro      3        0
Man City D    Walker      1        0
         D    Stones      0        0
         F     Jesus      1        0
         M     Foden      0        0

In [7]:
# Now let's ask for all the defenders on Man City.
prem.loc[('Man City', 'D'), :]

player  goals  assists
team     pos                        
Man City D    Walker      1        0
         D    Stones      0        0

No warnings. 

### Partial indexing
With the indexes set, we can easily subset the data using only one of the indexes. In pandas, this is called *partial indexing* because we are only using part of the index to subset identify the data we want. 


We  use the `xs()` method of DataFrame. Here we specify which level we are looking into. Note that I can reference the levels either by an integer or by its name.

In [8]:
# Get all of the 'Chelsea' observations.
print(prem.xs('Chelsea', level = 0), '\n')              

# Get all of the 'Man City' observations.
print(prem.xs('Man City', level = 'team'))

     player  goals  assists
pos                        
D    Cahill      0        0
F     Pedro      3        0 

     player  goals  assists
pos                        
D    Walker      1        0
D    Stones      0        0
F     Jesus      1        0
M     Foden      0        0


With `xs()`, we can partially index on the 'inner index' as well. Suppose we want all the defenders, regardless of team.

In [9]:
prem.xs('D', level='pos')

,player,goals,assists
team,,,
Chelsea,Cahill,0,0
Man City,Walker,1,0
Man City,Stones,0,0


As with a single index, we can get rid of the multIndex and replace it with a generic list of integers. This adds the index levels back into the DataFrame as columns. 

In [10]:
prem.reset_index(inplace=True)    # this moves the indexes back to columns
prem

,team,pos,player,goals,assists
0,Chelsea,D,Cahill,0,0
1,Chelsea,F,Pedro,3,0
2,Man City,D,Walker,1,0
3,Man City,D,Stones,0,0
4,Man City,F,Jesus,1,0
5,Man City,M,Foden,0,0


Who says two indexes are enough...let's try three levels of indexes!

In [11]:
prem.set_index(['team', 'pos', 'player'], inplace=True)
prem

goals  assists
team     pos player                
Chelsea  D   Cahill      0        0
         F   Pedro       3        0
Man City D   Walker      1        0
             Stones      0        0
         F   Jesus       1        0
         M   Foden       0        0

#### A multiIndex in columns
There is nothing that says you can't have multiple indexes in the `axis=1` dimension. Here is quick way to see this: transpose the DataFrame.

In [12]:
# .transpose swaps the rows for columns.
prem = prem.transpose()           
                    
prem

team    Chelsea       Man City                   
pos           D     F        D            F     M
player   Cahill Pedro   Walker Stones Jesus Foden
goals         0     3        1      0     1     0
assists       0     0        0      0     0     0

Notice that now 'goals' and 'assists' are values in the row index.

In [13]:
prem.loc['goals', ('Man City', 'D', 'Stones')]

0

In [14]:
prem.loc['goals', ('Man City', 'F', 'Jesus')]

1

Now the rows are named 'goals' and 'assists' and the columns are ('team', 'player', 'pos'). I'm not sure this is a very useful way to look at this particular dataset, but multiIndex columns can come in handy. \[Transpose is handy, too.\] Let's change it back.

In [15]:
prem = prem.transpose()
prem

goals  assists
team     pos player                
Chelsea  D   Cahill      0        0
         F   Pedro       3        0
Man City D   Walker      1        0
             Stones      0        0
         F   Jesus       1        0
         M   Foden       0        0

### Summary statistics by level
MultiIndexes provide a quick way to summarize data. We will see many different ways to do this &mdash; getting statistics by groups &mdash; and not all will involve a multiIndex. 

In [16]:
# Compute average goals for Chelsea players.
# We need to subset by the upppermost level.
print('Chelsea avg. goals', prem.xs('Chelsea', level='team')['goals'].mean())   

Chelsea avg. goals 1.5


In [17]:
# Compute average goals for defense. 
# We need to subset on the 'pos' level.
print('Defender avg. goals {0:.2f}.'.format(prem.xs('D', level='pos')['goals'].mean()))  

Defender avg. goals 0.33.


In [18]:
prem.xs('D', level='pos')['goals'].mean()

0.3333333333333333

In [19]:
prem.xs('Chelsea', level='team')

,,goals,assists
pos,player,,
D,Cahill,0,0
F,Pedro,3,0


Notice the syntax with xs.
```python
 prem.xs('Chelsea', level='team')['goals']
```

The `prem.xs('Chelsea', level='team')` is returning a DataFrame with all the columns. \[Try it!\]

We then use the usual square-bracket syntax to pick off just the column 'goals' and then hit with `.mean()`.


### Saving multiIndex DataFrames

Saving a multiIndexed DataFrame works like before. Pandas fills in all the repeated labels, so the output is ready to go. Run the following code and then open the csv files.

In [20]:
# Multiple indexes on rows
prem.to_csv('prem.csv')

# Multiple indexes on columns
prem = prem.transpose()
prem.to_csv('prem_transposed.csv')

## Practice

Let's take data from the [Current Population Survey](https://www.census.gov/programs-surveys/cps.html), which surveys about 60,000 households each month. We will compute some average wages. We will need to clean up a bit, then work with a multiIndex. Think of this as a mini-project.

1. Load the march cps data, 'CPS_March_2016.csv'.  Note: the missing values are '.'

In [21]:
CPS = pd.read_csv('CPS_March_2016.csv', na_values = '.')


2. Keep only those with `fulltimely == 1`
3. Keep only those with `5 <= hrwage <= 200`

In [29]:
CPS = CPS [ CPS['fulltimely'] ==1 ]
CPS = CPS [ CPS['hrwage'] >=5 ]
CPS = CPS [ CPS['hrwage'] <=200 ]
CPS.sample(4)

,hrwage,educ,female,fulltimely
55896,13.513514,HS diploma/GED,1,1.0
140148,30.288462,Less than HS,0,1.0
76794,16.346153,Less than HS,0,1.0
48754,14.182693,Graduate degree,1,1.0


In [30]:
#CPS = CPS.loc[ CPS['fulltimely'] == 1] 
#CPS = CPS.loc[( CPS['hrwage']<=200) & (CPS['hrwage']>=5) ] 

4. Rename 'female' to 'gender'
5. In column 'gender' replace 0 with 'male' and 1 with 'female'. Try using the the `.replace()` method of DataFrame [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html).

In [31]:
CPS = CPS.rename(columns = {'female':'gender'})


In [32]:
CPS['gender'] = CPS['gender'].replace({0:'male', 1:'female'}) 

In [33]:
#6
CPS.set_index(['gender','educ'], inplace = True)


6. Set the index to 'gender' and 'educ', in that order.
7. Sort the index. 

In [34]:
#7
CPS= CPS.sort_index(axis=0) 

In [35]:
CPS.head(10)

hrwage  fulltimely
gender educ                                 
female College degree  18.367348         1.0
       College degree  22.349272         1.0
       College degree  18.269230         1.0
       College degree  43.269230         1.0
       College degree  19.230770         1.0
       College degree  22.596153         1.0
       College degree  29.807692         1.0
       College degree  22.596153         1.0
       College degree  15.277778         1.0
       College degree  18.571428         1.0

8. Report the average wage for males and females. Try it with the `loc[]` method. 

In [36]:
#8
malemean = CPS.loc['male', 'hrwage'].mean()
femalemean = CPS.loc['female', 'hrwage'].mean()


print(malemean)
print(femalemean)

28.31492149779008
22.748034577703258


9. Report the average wage for `HS diploma/GED` and for `College degree`, regardless of gender. Use the `xs()` method.  

In [37]:
#9

hs_mean = CPS.xs('HS diploma/GED', level = 'educ')['hrwage'].mean()

college_mean = CPS.xs('College degree',  level = 'educ')['hrwage'].mean()

print(college_mean)
print(hs_mean)

31.963212864285698
19.107888095188635
